# Introduction to Keras 

![logo](https://keras.io/img/logo-small.png)

Based on the Keras Intro of [fchollet](https://twitter.com/fchollet)<br>


## Setup


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


ModuleNotFoundError: No module named 'tensorflow'

**INSTALL NOTES:**
* Keras will work out of the box in Coloab
* if you want to install it on your local machine, things can become complicated. Here are some general tips (no individual support provided!):
 

1.   You need GPU with >= 4GB mem, reacent driver and CUDA >= 9 up and running (this alone can be difficult)   
2.   DO NOT install *pip install keras* !!! this will give you an old, outdated version. Keras is now a submodule of Tensorflow (TF) - so you need to install Tesorflow >=1.12 or Tensorlow 2
3.   in most cases it works west to use anconda to install TF -> https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/





## Introduction

This guide will serve
as your first introduction to core Keras API concepts:


- How to prepare you data before training a model (by turning it into either NumPy
 arrays or `tf.data.Dataset` objects).
- How to do data preprocessing, for instance feature normalization or vocabulary
 indexing.
- How to build a model that turns your data into useful predictions,
using the Keras Functional API.
- How to train your model with the built-in Keras `fit()` method, while being
mindful of checkpointing, metrics monitoring, and fault tolerance.
- How to evaluate your model on a test data and how to use it for inference on new data.
- How to refine your model through hyperparameter tuning.

At the end of this guide, you will get pointers to end-to-end examples to solidify
 these concepts:

- Image classification
- Credit card fraud detection

For the next steps after this intro, consult the Keras API refence: https://keras.io/api/  



## Data loading & preprocessing

Neural networks don't process raw data, like text files, encoded JPEG image files, or
 CSV files. They process **vectorized** & **standardized** representations.

- Text files needs to be read into string tensors, then split into words. Finally, the
 words need to be indexed & turned into integer tensors.
- Images need to be read and decoded into integer tensors, then converted to floating
 point and normalized to small values (usually between 0 and 1).
- CSV data needs to be parsed, with numerical features converted to floating point
 tensors and categorical features indexed and converted to integer tensors.
Then each feature typically needs to be normalized to zero-mean and unit-variance.
- Etc.

Let's start with data loading.

## Data loading

Keras models accept three types of inputs:

- **NumPy arrays**, just like Scikit-Learn and many other Python-based libraries. This
 is a good option if your data fits in memory.
- **[TensorFlow `Dataset` objects](https://www.tensorflow.org/guide/data)**. This is a
high-performance option that is more suitable for datasets that do not fit in memory
 and that are streamed from disk or from a distributed filesystem.
- **Python generators** that yield batches of data (such as custom subclasses of
the `keras.utils.Sequence` class).

Before you start training a model, you will need to make your data available as one of
these formats. If you have a large dataset and you are training on GPU(s), consider
using `Dataset` objects, since they will take care of performance-critical details,
 such as:

- Asynchronously preprocessing your data on CPU while your GPU is busy, and bufferring
 it into a queue.
- Prefetching data on GPU memory so it's immediately available when the GPU has
 finished processing the previous batch, so you can reach full GPU utilization.

Keras features a range of utilities to help you turn raw data on disk into a `Dataset`:

- `tf.keras.preprocessing.image_dataset_from_directory` turns image files sorted into
 class-specific folders into a labeled dataset of image tensors.
- `tf.keras.preprocessing.text_dataset_from_directory` does the same for text files.

In addition, the TensorFlow `tf.data` includes other similar utilities, such as
`tf.data.experimental.make_csv_dataset` to load structured data from CSV files.

**Example: obtaining a labeled dataset from image files on disk**

Supposed you have image files sorted by class in different folders, like this:

```
main_directory/
...class_a/
......a_image_1.jpg
......a_image_2.jpg
...class_b/
......b_image_1.jpg
......b_image_2.jpg
```

Then you can do:

```python
# Create a dataset.
dataset = keras.preprocessing.image_dataset_from_directory(
  'path/to/main_directory', batch_size=64, image_size=(200, 200))

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64, 200, 200, 3)
   print(data.dtype)  # float32
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32
```

The label of a sample is the rank of its folder in alphanumeric order. Naturally, this
 can also be configured explicitly by passing, e.g.
`class_names=['class_a', 'class_b']`, in which cases label `0` will be `class_a` and
 `1` will be `class_b`.

**Example: obtaining a labeled dataset from text files on disk**

Likewise for text: if you have `.txt` documents sorted by class in different folders,
 you can do:

```python
dataset = keras.preprocessing.text_dataset_from_directory(
  'path/to/main_directory', batch_size=64)

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64,)
   print(data.dtype)  # string
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32
```




## Data preprocessing with Keras

Once your data is in the form of string/int/float NumpPy arrays, or a `Dataset` object
 (or Python generator) that yields batches of string/int/float tensors,
it is time to **preprocess** the data. This can mean:

- Tokenization of string data, followed by token indexing.
- Feature normalization.
- Rescaling the data to small values (in general, input values to a neural network
should be close to zero -- typically we expect either data with zero-mean and
 unit-variance, or data in the `[0, 1]` range.

### The ideal machine learning model is end-to-end

In general, you should seek to do data preprocessing **as part of your model** as much
as possible, not via an external data preprocessing pipeline. That's because external
data preprocessing makes your models less portable when it's time to use them in
production. Consider a model that processes text: it uses a specific tokenization
algorithm and a specific vocabulary index. When you want to ship your model to a
mobile app or a JavaScript app, you will need to recreate the exact same preprocessing
setup in the target language. This can get very tricky: any small discrepancy between
the original pipeline and the one you recreate has the potential to completely
 invalidate your model, or at least severely degrade its performance.

It would be much easier to be able to simply export an end-to-end model that already
includes preprocessing. **The ideal model should expect as input something as close as
possible to raw data: an image model should expect RGB pixel values in the `[0, 255]`
range, and a text model should accept strings of `utf-8` characters.** That way, the
 consumer of the exported model doesn't have
to know about the preprocessing pipeline.

### Using Keras preprocessing layers

In Keras, you do in-model data preprocessing via **preprocessing layers**. This
 includes:

- Feature normalization via the `Normalization` layer
- Image rescaling, cropping, or image data augmentation

The key advatange of using Keras preprocessing layers is that **they can be included
 directly into your model**, either during training or after training,
which makes your models portable.

Some preprocessing layers have a state:

- `TextVectorization` holds an index mapping words or tokens to integer indices
- `Normalization` holds the mean and variance of your features

The state of a preprocessing layers is obtained by calling `layer.adapt(data)` on a
 sample of the training data (or all of it).



**Example: normalizing features**


In [2]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)

normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))


var: 1.0000
mean: 0.0000


**Example: rescaling & center-cropping images**

Both the `Rescaling` layer and the `CenterCrop` layer are stateless, so it isn't
 necessary to call `adapt()` in this case.


In [3]:
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

cropper = CenterCrop(height=150, width=150)
scaler = Rescaling(scale=1.0 / 255)

output_data = scaler(cropper(training_data))
print("shape:", output_data.shape)
print("min:", np.min(output_data))
print("max:", np.max(output_data))


shape: (64, 150, 150, 3)
min: 0.0
max: 1.0


## Building models with the Keras Functional API

A "layer" is a simple input-output transformation (such as the scaling &
center-cropping transformations above). For instance, here's a linear projection layer
 that maps its inputs to a 16-dimensional feature space:

```python
dense = keras.layers.Dense(units=16)
```

A "model" is a directed acyclic graph of layers (***NOTE: Keras' simplyfied API can not handle cyclic graphs, I you need this, you need to use lower level Tensorflow***) . You can think of a model as a
"bigger layer" that encompasses multiple sublayers and that can be trained via exposure
 to data.

The most common and most powerful way to build Keras models is the Functional API. To
build models with the Functional API, you start by specifying the shape (and
optionally the dtype) of your inputs. If any dimension of your input can vary, you can
specify it as `None`. For instance, an input for 200x200 RGB image would have shape
`(200, 200, 3)`, but an input for RGB images of any size would have shape `(None,
 None, 3)`.


In [0]:
# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(None, None, 3))


After defining your input(s), you chain layer transformations on top of your inputs,
 until your final output:


In [0]:
from tensorflow.keras import layers

# Center-crop images to 150x150
l1 = CenterCrop(height=150, width=150)(inputs)
# Rescale images to [0, 1]
l2 = Rescaling(scale=1.0 / 255)(l1)

# Apply some convolution and pooling layers
l3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(l2)
l4 = layers.MaxPooling2D(pool_size=(3, 3))(l3)
l5 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(l4)
l6 = layers.MaxPooling2D(pool_size=(3, 3))(l5)
l7 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(l6)

# Apply global average pooling to get flat feature vectors
l8 = layers.GlobalAveragePooling2D()(l7)

# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(l8)


***NOTE: in Keras "fully connected", aka matrix multiplication layers are called "dense layers"***

Once you have defined the directed acyclic graph of layers that turns your input(s) into
 your outputs, instantiate a `Model` object:


In [0]:
model = keras.Model(inputs=inputs, outputs=outputs)


This model behaves basically like a bigger layer. You can call it on batches of data, like
 this:


In [8]:
data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")
processed_data = model(data)
print(processed_data.shape)


(64, 10)


You can print a summary of how your data gets transformed at each stage of the model.
 This is useful for debugging.

Note that the output shape displayed for each layers includes the **batch size**. Here
 the batch size is None, which indicates our model can process batchs of any size.


In [9]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
center_crop_2 (CenterCrop)   (None, 150, 150, 3)       0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0     

The Functional API also makes it easy to build models that have multiple inputs (for
instance, an image *and* its metadata) or multiple outputs (for instance, predicting
the class of the image *and* the likelihood that a user will click on it). For a
 deeper dive into what you can do, see our
[guide to the Functional API](/guides/functional_api/).


## Training models with `fit()`

At this point, you know:

- How to prepare your data (e.g. as a NumPy array or a `tf.data.Dataset` object)
- How to build a model that will process your data

The next step is to train your model on your data. The `Model` class features a
built-in training loop, the `fit()` method. It accepts `Dataset` objects, Python
 generators that yield batches of data, or NumPy arrays.

Before you can call `fit()`, you need to specify an optimizer and a loss function (we
 assume you are already familiar with these concepts). This is the `compile()` step:

```python
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())
```

Loss and optimizer can be specified via their string identifiers (in this case
their default constructor argument values are used):


```python
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
```

Once your model is compiled, you can start "fitting" the model to the data.
Here's what fitting a model looks like with NumPy data:

```python
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)
```

Besides the data, you have to specify two key parameters: the `batch_size` and
the number of epochs (iterations on the data). Here our data will get sliced on batches
 of 32 samples, and the model will iterate 10 times over the data during training.

Here's what fitting a model looks like with a dataset:

```python
model.fit(dataset_of_samples_and_labels, epochs=10)
```

Since the data yielded by a dataset is expect to be already batched, you don't need to
 specify the batch size here.




## Example: MLP for MNIST

In [10]:
# Get the data as Numpy arrays
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Build a simple model
inputs = keras.Input(shape=(28, 28))
l1 = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
l2 = layers.Flatten()(l1)
l3 = layers.Dense(128, activation="tanh")(l2)
l4 = layers.Dense(128, activation="tanh")(l3)
outputs = layers.Dense(10, activation="softmax")(l4)
model = keras.Model(inputs, outputs)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

# Train the model for 1 epoch using a dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
print("Fit on Dataset")
history = model.fit(dataset, epochs=1)


11493376/11490434 [==============================] - 0s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
rescaling_3 (Rescaling)      (None, 28, 28)            0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainabl

The `fit()` call returns a "history" object which records what happened over the course
of training. The `history.history` dict contains per-epoch timeseries of metrics
values (here we have only one metric, the loss, and one epoch, so we only get a single
 scalar):


In [11]:
print(history.history)


{'loss': [0.1385771930217743]}


For a detailed overview of how to use `fit()`, see the
[guide to training & evaluation with the built-in Keras methods](
  /guides/training_with_built_in_methods/).


### Keeping track of performance metrics

As you're training a model, you want to keep of track of metrics such as classification
accuracy, precision, recall, AUC, etc. Besides, you want to monitor these metrics not
 only on the training data, but also on a validation set.

**Monitoring metrics**

You can pass a list of metric objects to `compile()`, like this:



In [12]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
history = model.fit(dataset, epochs=1)


938/938 [==============================] - 3s 3ms/step - loss: 0.0970 - acc: 0.9706


**Passing validation data to `fit()`**

You can pass validation data to `fit()` to monitor your validation loss & validation
 metrics. Validation metrics get reported at the end of each epoch.


In [13]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
history = model.fit(dataset, epochs=1, validation_data=val_dataset)


938/938 [==============================] - 3s 4ms/step - loss: 0.0676 - acc: 0.9796 - val_loss: 0.0912 - val_acc: 0.9711


### After `fit()`: evaluating test performance & generating predictions on new data

Once you have a trained model, you can evaluate its loss and metrics on new data via
 `evaluate()`:


In [14]:
loss, acc = model.evaluate(val_dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)


157/157 [==============================] - 0s 2ms/step - loss: 0.0912 - acc: 0.9711
loss: 0.09
acc: 0.97


You can also generate NumPy arrays of predictions (the activations of the output
 layer(s) in the model) via `predict()`:


In [15]:
predictions = model.predict(val_dataset)
print(predictions.shape)


(10000, 10)


## Debugging your model with eager execution

If you write custom training steps or custom layers, you will need to debug them. The
debugging experience is an integral part of a framework: with Keras, the debugging
 workflow is designed with the user in mind.

By default, your Keras models are compiled to highly-optimized computation graphs that
deliver fast execution times. That means that the Python code you write (e.g. in a
custom `train_step`) is not the code you are actually executing. This introduces a
 layer of indirection that can make debugging hard.

Debugging is best done step by step. You want to be able to sprinkle your code with
`print()` statement to see what your data looks like after every operation, you want
to be able to use `pdb`. You can achieve this by **running your model eagerly**. With
 eager execution, the Python code you write is the code that gets executed.

Simply pass `run_eagerly=True` to `compile()`:

```python
model.compile(optimizer='adam', loss='mse', run_eagerly=True)
```

Of course, the downside is that it makes your model significantly slower. Make sure to
switch it back off to get the benefits of compiled computation graphs once you are
 done debugging!

In general, you will use `run_eagerly=True` every time you need to debug what's
 happening inside your `fit()` call.


## Finding the best model configuration with hyperparameter tuning

Once you have a working model, you're going to want to optimize its configuration --
architecture choices, layer sizes, etc. Human intuition can only go so far, so you'll
 want to leverage a systematic approach: hyperparameter search.

You can use
[Keras Tuner](https://keras-team.github.io/keras-tuner/documentation/tuners/) to find
 the best hyperparameter for your Keras models. It's as easy as calling `fit()`.

Here how it works.

First, place your model definition in a function, that takes a single `hp` argument.
Inside this function, replace any value you want to tune with a call to hyperparameter
 sampling methods, e.g. `hp.Int()` or `hp.Choice()`:

```python
def build_model(hp):
    inputs = keras.Input(shape=(784,))
    x = layers.Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu'))(inputs)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model
```

The function should return a compiled model.

Next, instantiate a tuner object specifying your optimiation objective and other search
 parameters:


```python
import kerastuner

tuner = kerastuner.tuners.Hyperband(
  build_model,
  objective='val_loss',
  max_epochs=100,
  max_trials=200,
  executions_per_trial=2,
  directory='my_dir')
```

Finally, start the search with the `search()` method, which takes the same arguments as
 `Model.fit()`:

```python
tuner.search(dataset, validation_data=val_dataset)
```

When search is over, you can retrieve the best model(s):

```python
models = tuner.get_best_models(num_models=2)
```

Or print a summary of the results:

```python
tuner.results_summary()
```


## End-to-end examples

To familiarize yourself with the concepts in this introduction, see the following
 end-to-end examples:
- [Image classification](/examples/vision/image_classification_from_scratch/)
- [Credit card fraud detection](/examples/structured_data/imbalanced_classification/)


## What to learn next

- Learn more about the
[Functional API](/guides/functional_api/).
- Learn more about the
[features of `fit()` and `evaluate()`](/guides/training_with_built_in_methods/).
